In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
data=pd.read_csv("../input/records/records.csv")
all_text=data["transcription_text"]

In [3]:
data.shape

(35168, 16)

In [4]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punctuation
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s
start=time.time()
for i in tqdm(range(35168)):
    all_text[i] = process(all_text[i])
for i in tqdm(range(35168)):
    all_text[i] = " ".join(all_text[i])
end=time.time()
print(end-start)

100%|██████████| 35168/35168 [00:19<00:00, 1766.87it/s]

119.65206360816956


In [5]:
all_text

0        one hundred dollars . I never heard Bagby comp...
1        The Freedmens Bureau vs. B.M . Richardson Assa...
2        The Freedmens Bureau vs. B. M. Richardson , Co...
3        one hundred dollars , two suits of clothes wou...
4        questions , I gave answer , I whipped.— All I ...
                               ...                        
35163    648 Z Date.Name of Male.Place of Residence.Nam...
35164    650 Date . Name of Male . Name of Female . Pla...
35165    Sarah Brame Died June 25th 1847 . 1866 1847 De...
35166    Frances S. Benners Born August 30th 1795 Died ...
35167    State of North Carolina Bladen County Alfred ....
Name: transcription_text, Length: 35168, dtype: object

****

****

In [6]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 288.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 1.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.1-py3-none-any.whl size=125774 sha256=9145c35c43c64596e96f71fa636a5afac19b303893b1dd5088baa6c776575b76
  Stored in directory: /root/.cache/pip/wheels/58/27/2f/708b4f002c226e57b6243769da345c650633175c7634f93365
Successfully built sentence-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.7.0
    Uninstalling huggingface-hub-0.7.0:
      Successfully uninstalled huggingface-hub-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cached-path 1.1.3 requires huggingface-hub<0.8.0,>=0.0.12, but you have huggingface-hub 0.8.1 which i

In [7]:
def num_of_element(clust_assign, clus_sent) :
    dic={}
    L=list(clust_assign)
    for i in range(len(clus_sent )):
        dic[i]=L.count(i)
    
    return dic
        

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(all_text)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1099 [00:00<?, ?it/s]

In [9]:
embeddings.shape

(35168, 384)

In [10]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

258.7842833995819


In [11]:
len(clustered_sentences)

309

In [12]:
cluster_assignment

array([ 58,  58,  58, ..., 125, 125, 186])

In [13]:
dic1=num_of_element(cluster_assignment,clustered_sentences)

In [14]:
dic1[0]

153

In [15]:
len(clustered_sentences[250])

66

In [16]:
clustered_sentences[0][:15]

['Charges Specifications preferred Second Lieutenant James H. Griggs Thirty Seventh Regiment United States Colored Troops . Charge Misapplication of public money intrusted payment of men command . Specification 1st In Second Lieutenant James H. Griggs . Thirty Seventh Regiment United States Colored Troops , officer Service of United States Acting Assistant Quartermaster Post of Fort Macon North Carolina , command one Bonaparte Fulford citizen employment of United States first fifth day of July A.D. 1866 , first twentieth day of August A.D. 1866 , inclusion compensation of forty dollars per month , furnished Brevet Lieutenant Colonel P.D . Stubbs Captain Assistant Quartermaster United States Volunteers , Cheif Quartermaster Military Commander of North Carolina twelfth day of November A.D. 1866 , check number payable order Assistant Treasury New York sum of thirty three dollars thirty two cents purpose of paying said Bonaparte Fulford',
 'lawfull wages period herein Specified withold fai

In [17]:
clustered_sentences[300][:50]

['No 22 A Miss Alice Ashley per Gabriel Davis Attorney I.B . 36 A ACSC GA 65 Applies restoration of property Decb 9 65 Restored',
 'A 29 Alston W.A Application Plantation Georgtn Dist A 29 Hd Qr Acty Sub Asst Comr Bureau of R.F . AL . Georgetown S.C. Feby 8th 1866 Respectfully forwarded recommended B.F. Smith Bvt Lt Col U.S.A. Actg Sub Asst Commr B of R.F.AL . Feby 20 66 Restored',
 'B No 19 W Baggett James . W. Charleston So Ca Applies Restoration of cotton plantation Prince Williams Parish Beaufort Dist So . Ca . property I.B . 52 B Asst Com SC Ga 65 Thro William Whaley Esqre of Charleston So Ca Oct 11 1815 order issued restoration Restored R.S . Recd AAGO Asst Com SC Ga Oct 1165',
 'No : 15 B Daniel Blake Application Restoration of Real Personal Property . Palmetto Hill Combahee River I.B . 41 B Asst Com SC Ga 65 Restored RS Oct 6 , 1865 . Order issued restoration of property . Recd AAGO Asst Com SC Ga Oct 5 , 1865',
 'No 11 B Daniel Blake Application Restoration of Real Personal Pr

In [18]:
clustered_sentences[200][:]

['Bastrop Bastrop Co. Texas , March 28 , 1867 I F. Laake , agent trustees of church building owned German Methodist Episcopal Church South , Bastrop Texas , hereby agree contract rent said building school purposes , Bureau Refugees Freedmen Abandoned Lands , State of Texas , three months , beginning 1st day of April 1867 ending 30th day of June 1867 , monthly rental of fifteen Dollars currency , paid monthly ; Bu . of R.F . A.L . agreeing become responsible damage scholars may said building enclosure time Frany Laake Witness Geo . M. Decherd To Supt . of Schools Bu . R.F . A.L State of Texas',
 'O.S . F.J. Reed . June 767P.U . Bastrop , June 367 Porter Byron Sub Asst Comr Enclose Miss M E Wilder School report May 1867 . States Soloman Fish resident of Sinbin Bastrop Co agreed give lot of ground timber enough put school House , The Lot given freedman corporation of Bastrop selected of stone foundation ground hopes open School next October , Will ask assistance exhausted extracted , Cold

In [19]:
clustered_sentences[250][:]

['2 Complaints 1868 Date County Names of Parties Notice of Complaint Action Taken Remarks Jany 8 Franklin Geo N Macon vs Frank . Bob Ned Dissatisfaction settlement of ac , crop made 1867 Jany 868 ac settled See T.R . p 66 Jany 8 Franklin Bill Guplin vs Joe Foster Deft took carried away 2 bbls of corn Feby67 . Falsely claiming plff owed Jany 868 . Plff requests proceeding stayed Court Jany 9 Granville DE I. Baskerville vs Aaron Joe Lyne Refusal of Defts certain work expressly stipulated contract 1867 Summons issued returnable Jany 27 11 Judgment Jany 27 Deft killed plffs steer July67 hard driving business Deft pay 10 damages Jany 9 Granville James A Bullock vs William Jeffreys Refusal strip tobacco made 1867 Summons issued returnable Jany 27 11 No appearance Jany 9 Franklin Bob Hunt vs Frank Foster F Deft hired Plffs daughter year 1867 refuses pay amt 25 Jany 968 . On settlement of acs Mason Frank others bal due Frank 15.27 . Ordered Mason pay amt Bob , 7 children , Frank none See L.B .

****

# all_mpnet_base_v2 , threshold=3.5

In [20]:
model_1 = SentenceTransformer('all-mpnet-base-v2')
embeddingss = model_1.encode(all_text)
embeddingss.shape

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1099 [00:00<?, ?it/s]

(35168, 768)

In [21]:
embeddingss.shape

(35168, 768)

In [22]:
start=time.time()
# Normalize the embeddings to unit length
embeddingss = embeddingss /  np.linalg.norm(embeddingss, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddingss)
cluster_assignment_1 = clustering_model.labels_

clustered_sentences_1 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment_1):
    if cluster_id not in clustered_sentences_1:
        clustered_sentences_1[cluster_id] = []

    clustered_sentences_1[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

376.9602167606354


In [23]:
len(clustered_sentences_1)

245

In [24]:
dic_1=num_of_element(cluster_assignment_1,clustered_sentences_1)

In [25]:
cluster_assignment_1

array([ 53, 140, 140, ..., 179,  24, 147])

In [26]:
len(clustered_sentences_1[180])

22

In [27]:
clustered_sentences_1[0][:]

['B . North Carolina Columbus County This day came Calvin E. Gore , justice of Peace said County . William Q. Maultsby , sworn saith oath well acquainted Ben Simpson freedman , known past seven eight years , always bad character , generally regard person of untruth , vicious disposition , could believe oath , would regard dangerous character community . He swears , known Benjamin M. Richardson six years , known peaceable quiet moral man , of kind disposition , employ time negroes slavery , freedmen service since liberation , always treated uniform kindness ; regarded community worthy citizen . He swears Mr Richardson politics opposed war , took active part encouraging rebellion , influence discourage ; war man ; point of humanity helped poor needy , without regard race color W. Q. Maultsby seal Signed Sealed Sworn This March 19th 1866 , Calvin C. Gore J.P. seal',
 'C North Carolina Columbus County This day came Calvin G. Gore , justice of peace said County . J Lloyd McKay sworn saith o

In [28]:
clustered_sentences_1[200][:20]

['NARA 128 98 No . 24 LEASE FOR HOUSES AND TENEMENTS . Memorandum of Agreement made 12th day of April 1865 , D. Heaton Supg . Special Agent of Treasury Department , duly appointed Acts of Congress respectively approved March 12 , 1863 , July 2 , 1864 , taking charge of captured abandoned property , leasing abandoned confiscable lands , houses , tenements Sixth Special Agency , Mrs. Mary R. Hooks of Wilmington County of New Hanover State of North Carolina WITNESSETH , pursuance of said Acts , of instructions of Secretary of Treasury , said Agent , behalf of United States , agrees upon terms hereinafter contained , lease said Mrs. Mary R. Hooks First day of April , 1865 , month month , either party hereto liberty terminate lease end of month date hereof , following described premises : House Cor . of Mulberry Third Sts . Block 205 Wilmington North Carolina And said Mrs. Mary R. Hooks hereby agrees pay said Agent One dollars per month , month date hereof , long shall continue possession o

In [29]:
clustered_sentences_1[150][:20]

['AsstSUBASSISTANT COMMISSIONERS MONTHLY REPORT Education of Freedmen Refugees SubDistrict , State of La charge of Month of Febry 1868 , 1 . Name of SubDistrict Avoyelles 2 . Whole number of Refugee Freedmens Schools District 3 Day 3 Night Sabbath 3 . Location of Schools Marksville Hydropolis Holmesville 4 . Whole number of Teachers 3 White 1 Colored 2 5 . Names postoffice address of DaySchool Teachers A Hitten Holmesville La Miss Tassin Marksville La 6 . Whole number of Schoolhouses Freedmen District Their condition , capacity , value , owned In Marksville . Mrs. Plumel Hydropolis Holmesville Houses furnished used ordinary Quarters 7 . Number of visits Schools 3 Day 3 Night Sabbath 8 . Number of educational meetings held month 1 Where Marksville 9 . Number names of places , destitute , DaySchools might organized none , teacher supported Mansura Miroville 10 . Number of pupils would attend Schools fifty 11 . Amount would probably raised Freedmen , school purposes , destitute neighborho

In [30]:
clustered_sentences_1[180][:]

['St Johns Nov .',
 'St James No.9 Nov',
 'St John No3 Nov',
 'St James No 2 Nov',
 'St John No 5 Nov',
 'St John No 4 Nov .',
 'St James No 13 Nov .',
 'St James No.1 Nov',
 'St James No.6 Nov',
 'St John No Dec .',
 'St. John No5 Dec ,',
 'St James No.2 Dec .',
 'St. James No 9 Dec .',
 'St James No.6 Dec .',
 'St James No.1 Dec .',
 'St John No.4 Dec',
 'St John No.3 Dec',
 'St James No 3 Dec 1st 15th Dec',
 'St John No Dec',
 'St John No 3 Dec',
 'St James No 2 Dec',
 'St James No 3 Dec']

****

# all-MiniLM-L6-v2 , threshold=2

In [31]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment2 = clustering_model.labels_

clustered_sentences2 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment2):
    if cluster_id not in clustered_sentences2:
        clustered_sentences2[cluster_id] = []

    clustered_sentences2[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

255.44524550437927


In [32]:
len(clustered_sentences2)

1008

In [33]:
dic2=num_of_element(cluster_assignment2,clustered_sentences2)

In [34]:
dic2[0]

307

In [35]:
len(clustered_sentences2[1007])

31

In [36]:
clustered_sentences2[0][:10]

['L.W . Stevenson , Supt of Education I would observe information desire remain division of county yr. Am near Seven months teaching , affected , respectable improvement among Pupils . Respectfully H. Ames ED FORM , NO . 3 TEACHER ’ S MONTHLY SCHOOL REPORT For month of May , 1870 District of County , Guadaloupe State , Tex Name of School , High Hill Grammar School Name of Teacher , Hugh Ames Forwarded Mr. Andrew Wade , RECD OFFICE B.R.F . A.L . RECD JUN 5 1870 Explanations If of within questions answered Teacher , may left Superintendent . Place answer , extent of knowledge opposite question , fill remaining spaces . Night Sabbath Schools taught group of Teachers reported . In Remarks notice important fact blank call . It hoped Educational Societies adopt form of Report ; , Bureau furnish blanks . Should extended forms used , would oblige Commissioner include items of . Bureau R.F . A.L Washington . D.C.. October 1 . 1868',
 'Teachers Monthly School Report . For month of March 1866 . D

In [37]:
clustered_sentences2[1][:20]

['Invoice of Tuition Tickets sold Anora Killgrove , Principal Teacher Abraham Lincoln School , 1st 25th day of July 1866 , transfer of blank tickets cash arising sale of tickets E.S . Stoddard Director . DATE TICKETS . DOLLS . CTS . July 1st Total amount of blank tickets received per invoice ... 100 July 12 Sold 1.50 ... 2 3 00 Sold 1.00 ... July 25 Sold less month , pro rata ... Entered folio 198 Tuition Ledger Total 2 3 00 RECAPITULATION Total amount of tickets sold 1.50 ... 76 117 00 Total amount of tickets sold 1.25 ... 7 8 75 Total amount of tickets sold 1.00 ... Total amount of tickets sold less month , pro rata Total tickets sold cash recd 83 122 754 BALANCE 17 I HEREBY CERTIFY , I 25th day of July New Orleans La . transferred E.S . Stoddard Director , amount of blank tickets cash stated . Anora Killgrove Principal . I hereby acknowledge I 25th day of July 1866 , NEW ORLEANS , LA. , received Anora Killgrove Principal of A Lincoln School , Three Dollars Cents cash seventeen unsol

In [38]:
clustered_sentences2[1000][:]

['BUR . REF. , FREEDMEN AND ABANDONED LANDS , State of Texas . SCHOOL REPORTS FOR THE MONTH OF January 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS , State of Texas . SCHOOL REPORTS FOR THE MONTH OF February 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF March 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF April 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF May 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF June 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS , State of Texas . SCHOOL REPORTS FOR THE MONTH OF July , 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONDED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF AUGUST , 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS , State of Texas . SCHOOL REPORTS FOR THE MONTH OF September , 1866 .',
 'BUR .

In [39]:
clustered_sentences2[1007][:]

['School Report of School No 1 County of City of Galveston For Month of October 1866 .',
 'SCHOOL REPORT of L. W. County of Galveston Texas For Month of November , 1886',
 'SCHOOL REPORT Of G. T Rudy County of Galveston For Month of November , 1866',
 'SCHOOL REPORT Of G T. Ruby County of Galveston For Month of December , 1866',
 'School Report of Galveston County of Galveston For Month of Jan. , 1867',
 'SCHOOL REPORT Of Galveston Texas County of Galveston For Month of February 1867',
 'SCHOOL REPORT OF Galveston COUNTY OF Galveston For Month of March 1867 . FORWARDED March 29 1867 . RECEIVED O.S . March 30th 1867 .',
 'Private School . SCHOOL REPORT OF Henry Gomez COUNTY OF Galveston For month of October FORWARDED Nov 1st 1867 . RECEIVED O.S . 1867 . Recd S.A.C . Nov 1st 67',
 'Private School . SCHOOL REPORT OF plant Medows COUNTY OF Galveston For month of october FORWARDED oct th 30 1867 . RECEIVED O.S . 1867 . Recd S.A.C . Nov 1st 67',
 'SCHOOL REPORT OF The State Texas COUNTY OF G

****

# all_mpnet_base_v2 , threshold=2

In [40]:

start=time.time()
# Normalize the embeddings to unit length
embeddingsqs = embeddingss /  np.linalg.norm(embeddingss, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddingss)
cluster_assignment_2 = clustering_model.labels_

clustered_sentences_2 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment_2):
    if cluster_id not in clustered_sentences_2:
        clustered_sentences_2[cluster_id] = []

    clustered_sentences_2[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)


375.2260649204254


In [41]:
len(clustered_sentences_2)

752

In [42]:
len(clustered_sentences_2[0])

72

In [43]:
clustered_sentences_2[0][:10]

['also depositions of T.N.F . Alston Israel Green George Davis Frank Hight Edmund Hawkins Arthur Vaughn All of respectfully submitted Chas . B. Whittemore Agent Bu . R. F. A. L. Net proceeds of sales 609.14 500 Staves hand worth 17.50 626.64 whole amount of a.c. Israel Green Geo . Davis 578.41 Less allowance Tools amount charged Fodder 42.32 476 09 Less 12 net proceeds sales 313.32 162.77 Less work plantation allowed Mr Alston 39.23 Balance due Mr. Alston 123.54 Israel Green Geo . Davis',
 'Deponent claims six dollars month four months worked Woodley John X mark Mason Sworn subscribed 21st day of May 1866 Tho H Hay Capt VRC Asst Supt',
 'Joshua Nunn George Wright Sept 1 1865 To four months work Blacksmith shop . 1st of May65 lst of Sept 18 per month 72.00 Credit S. Ayscues acct paid 3.50 68.50 Capt Th C. Hay Sir At request of George Wright I drawn account George formerly belonged aunt , He good plantation Smith worked Mr Nunn last year I doubt account correct , least regard time I know

In [44]:
clustered_sentences_2[500][:]

['Colored APPLICATION OF John Pearson Fanny Avery Dick Avery State of North Carolina Morganton June 15 1867 .',
 'Colored APPLICATION OF Clarissa Sudreth Kate Caldwell State of North Carolina Morganton June , 15 , 1867 .',
 'Colored APPLICATION OF Delia Scott Luncenda Walton State of North Carolina Morganton June 17 , 1867 .',
 'Colored APPLICATION OF Wash Johnson State of North Carolina Morgantin June 17 , 1867 .',
 'Colored APPLICATION OF Clarra Erwin Nathanal , Avery Hannah Gaylord State of North Carolina Morganton June 17 1867',
 'Colored APPLICATION OF Elea Smith State of North Carolina Morganten June 18 1867 .',
 'Colored Application of Lukey Pearson Nancey Hanson , Dinah Avery State of North Carolina Morganton June 23rd 1867 .',
 'Colored APPLICATION OF Wm Edwards George Presnell Pink Mc Entyre State of North Carolina Morganton June 24 1867 .',
 'Colored Application of Sandey Erwin George Pearson Chaney Tate State of North Carolina Morganton , June 25th 1867 .',
 'Colored Applic

In [45]:
clustered_sentences_2[501][:]

['O.S.F.S . Recvd Apl 2967 Palestine Apl 1067 Morrison J.H . S.A. Com Report Freedmen want of schools recommended Mr. Boyd teacher',
 'Palestine Apl 1067 J T Kirkman Lt 26 US In A A A Genl Supt of S Reports freedmen anxious school : one needed Palestine recommend Mr Pat Boyd Teacher . JH Morrison Sub Asst Com',
 'B.R.F . A.L . M33 May 1467 Palestine May 767 Morrison I.H . Sub Asst Com Reports one school Palestine children of Freedman of labor successful operation 68 scolhars . A suitable house purchased Freedman 600 . 200 paid paid 1st of Dec 1867 . The Colored people generally county subject of schools earnest consideration , determined take steps . School opened May 4th Office May 1467',
 'Office sub Asst Comr B.R.F A.L . Palestine May 7th 1867 Sir I report one school Palestine children of Freed people of Color successful operation fiftyeight schollars , A suitable house purchased Freedmen , six hundred dollars . Two hundred dollars paid paid 1st day of December 1867 . The colored pe

In [46]:
clustered_sentences_2[200][:]

['M 109 State of Texas , County of Smith Know men presents Joseph Smith of aforesaid State County of first part , Bvt Lieut Col D L Montgomery , Sub Asst . Com . B R F A.L . State of Texas , of Second part , hereby make enter following agreement . Witnesseth That I Joseph Smith of first part hereby agree give immediate possession of certain frame house situated South side of Public Square of Town of Tyler Smith Co. joining old Confederate drug store Sub Assistant Com . B.R.F A.L . school purposes , freedpeople consideration of payment of monthly rent of Seven dollars US . currency . That I , D.L . Montgomery , Bvt Lt Col U.S.A. Sub Asst Com . B.R.F . A Lands State of Texas of first part hereby agree Agent of United States Government pay Joseph Smith of second part owner proprietor of aforesaid building monthly rent of seven dollars US Currency . In witness whereof herewith subscribe names twelfth day of October 1867 J P. Smith D. L. Montgomery Bvt . Lt. Col US.A . Sub Asst . Com . Sign

In [47]:
clustered_sentences_2[700][:]

['SCHOOL REPORT Of San Antonio County of Bexar For Month of November , 1866',
 'School Report Of San Antonio County of Bexar For Month of Jan , 1867',
 'SCHOOL REPORT OF San Antonio COUNTY OF Bexar For Month of March 1867 . FORWARDED March 31 1867 . RECEIVED O.S . April 5th 1867 . Mr N. Whiting . San Antonio . Texas . School Report March . 1867 .',
 'SCHOOL REPORT Of San Antonio County of Bexar For Month of April 1867 Forwarded Apr 30 1867 . Received O.S . May 7th 1867 . Mr N. Whiting . San Antonio , Texas . School Report April , 1867 .',
 'W7BRc . S.a1867 . SCHOOL REPORT of San Antonio County of Bexar For Month of April 1867 . FORWARDED Apr 30 1867 . Recd O.S.A.C San Antonio May 1 , 1867 . RECEIVED O.S . ... ..1867 .',
 'SCHOOL REPORT OF San Antonio COUNTY OF Bexar For Month of June 1867 . FORWARDED July 1 1867 . RECEIVED O.S . July 10 1867 .',
 'SCHOOL REPORT OF Beaumont COUNTY OF Bexar For month of August FORWARDED September 2nd 1867 .1868 RECEIVED O.S . ... ... ... . 1867 .1868',
 

In [48]:
clustered_sentences_2[750][:]

['War Department . Bureau Refugees , Freedmen Abandoned Lands , In The Field . Charleston , S. C. , February 10th 1866 . J Evans Edings applicant restoration of tract Edisto Island known Edingsville bounded east Atlantic Ocean adjoining lands of Ephraim Baynard appearing said tract unoccupied either freedmen others except one family appears report held Bureau of Refugees , Freedmen Abandoned Lands , conformed requirements of Circular No . 15 , of said Bureau , dated Washington , D.C. , Sept. 12 , 1865 , aforesaid property hereby restored possession . The instrument considered null void unless obligation herewith attached subscribed said J Evans Edings faithfully fully complied . All differences arising instrument obligation adjudicated Board of Supervisors constituted Special Field Order No 1 Bureau Refugees , Freedmen Abandoned Lands , dated Charleston , October 19 , 1865 . Pursuant Instructions of President of United States . O.O . Howard , Major General , Commissioner . Official : A

In [49]:
clustered_sentences_2[751][:]

['SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For Month of April 1867 . FORWARDED ... ... 1867 . RECEIVED O.S . May 8th 1867 . Mr G.C . Booth . Richmond . Texas . School Report April . 1867 .',
 'SCHOOL REPORT OF ... COUNTY OF Fort Bend , Texas For month of May 1867 FORWARDED 3rd June 1867 . RECEIVED O.S . June 5th 1867 .',
 'SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For month of May 1867 FORWARDED May 31st 1867 RECEIVED O.S . ... ... .. 1867',
 'SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For month of June 1867 Forwarded ... 1867 . RECEIVED O.S . July 1st of 1867 .',
 'SCHOOL REPORT OF Mrs Mary J. Cole COUNTY OF Fort Bend For month of June FORWARDED August 28 1867 . RECEIVED O.S . Aug 29th 1867 .',
 'March 67 SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For month of July 1867 FORWARDED ... . 1867 . RECEIVED O.S . Sept 13 1867 .',
 'SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For month of October , 1867 . FORWARDED Nov 3 1867 . RECEIVED O.S . Nov 15th 1867 . Teacher A.M.A.S 

****

# all-MiniLM-L6-v2 , threshold=3

In [50]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment3 = clustering_model.labels_

clustered_sentences3 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment3):
    if cluster_id not in clustered_sentences3:
        clustered_sentences3[cluster_id] = []

    clustered_sentences3[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

253.89698028564453


In [51]:
len(clustered_sentences3)

407

In [52]:
dic3=num_of_element(cluster_assignment3,clustered_sentences3)

In [53]:
dic3[0]

92

****

# all-MiniLM-L6-v2 , threshold=4

In [54]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=4) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment4 = clustering_model.labels_

clustered_sentences4 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment4):
    if cluster_id not in clustered_sentences4:
        clustered_sentences4[cluster_id] = []

    clustered_sentences4[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

254.72584867477417


In [55]:
len(clustered_sentences4)

237

In [56]:
dic4=num_of_element(cluster_assignment4,clustered_sentences4)

In [57]:
dic4[0]

134

****

In [58]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment5 = clustering_model.labels_

clustered_sentences5 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment5):
    if cluster_id not in clustered_sentences5:
        clustered_sentences5[cluster_id] = []

    clustered_sentences5[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

254.13716769218445


In [59]:
len(clustered_sentences5)

146

In [60]:
dic5=num_of_element(cluster_assignment5,clustered_sentences5)